# INFO

To infer (generate), Colab free is enough even for float16 (both Tesla T4 & no GPU).

To train (finetune), you must will need at least Colab Pro or above. Remember to check if your GPU is A100. GPU memory > 20 GB is required.

生成的话，即使是原版 float16 精度，免费版 Colab 也跑得起来，包括 T4 显卡和无显卡。

要训练，至少要 Colab Pro 或 Pro+, 记得检查 GPU 是不是 A100，需要 显存 > 20 GB。

# Requirements

In [ ]:
!nvidia-smi

Sun Apr 16 15:45:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !apt update
# !apt install -qq -y nvidia-cuda-toolkit

In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


### 安装依赖

In [ ]:
!git clone https://github.com/ljsabc/Fujisaki
%cd Fujisaki
!sed -i 's/==/>=/g' requirements.txt
%pip install -q -r requirements.txt

fatal: destination path 'Fujisaki' already exists and is not an empty directory.
/content/Fujisaki
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


It's recommended that you prepare your data locally. Colab Pro is so expensive...

In [ ]:
!cp config_example.py config.py
!sed -i 's/PARSE_REPLIES = False/PARSE_REPLIES = True/g' config.py
import config
config.PARSE_REPLIES = True

In [ ]:
# If you prepare your data locally...
# !cp -a /content/drive/MyDrive/twitter/tweets.md .

# Train

In [ ]:
%cd Fujisaki

/content/Fujisaki


In [ ]:
!python ./tokenize_dataset_rows.py --json_path ./tweets.md --save_path tweets.tokens --max_seq_length 240

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Extracting data files: 100% 1/1 [00:00<00:00, 1393.46it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8f82e7fc8ee2271c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 250.96it/s]


In [ ]:
# !python finetune.py --dataset_path tweets.tokens --lora_rank 16 --per_device_train_batch_size 8 --gradient_accumulation_steps 1 --num_train_epoch 3 --save_steps 2000 --save_total_limit 2 --learning_rate 2e-4 --fp16 --remove_unused_columns false --logging_steps 50 --output_dir output --warmup_steps 100

Let's Train in the notebook

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training
from dataclasses import dataclass, field
import datasets
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

In [ ]:
from finetune import FinetuneArguments, CastOutputToFloat, data_collator, ModifiedTrainer

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, device_map='auto', revision=glm_rev)

In [ ]:
import json
with open('tweets.md', 'r', encoding='utf-8') as f:
    tweets = json.load(f)

tweets_count = len(tweets)
del tweets
tweets_count

9686

In [ ]:
finetune_args, training_args = HfArgumentParser(
    (FinetuneArguments, TrainingArguments)
).parse_args_into_dataclasses(
    ['--dataset_path', 'tweets.tokens',
     '--lora_rank', '16',
     '--per_device_train_batch_size', str(9700 // (40*45)),  # adjust yourself
     '--gradient_accumulation_steps', '1',
     '--num_train_epoch', '8',
     '--save_steps', '2000',
     '--save_total_limit', '2',
     '--learning_rate', '2e-4',
     '--remove_unused_columns', 'false',
     '--logging_steps', '50',
     '--output_dir', 'output',
     '--warmup_steps', '100',
     '--fp16'
]
)

In [ ]:
device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}

In [ ]:
model = AutoModel.from_pretrained(
    "THUDM/chatglm-6b", trust_remote_code=True, device_map=device_map, revision=glm_rev
)
# , load_in_8bit=True

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# model = prepare_model_for_int8_training(model)
#model.gradient_checkpointing_enable()
#model.enable_input_require_grads()
model.is_parallelizable = True
model.model_parallel = True
#model.lm_head = CastOutputToFloat(model.lm_head)
# model.config.use_cache = (
#     False  # silence the warnings. Please re-enable for inference!
# )

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=finetune_args.lora_rank,
    lora_alpha=32,
    lora_dropout=0.1,
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
dataset = datasets.load_from_disk(finetune_args.dataset_path)

In [ ]:
trainer = ModifiedTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
50,6.222600
100,4.269800
150,3.722900
200,3.677800
250,3.571700
300,3.548700
350,3.431100
400,3.604700
450,3.570700
500,3.441600


TrainOutput(global_step=15504, training_loss=1.8674790621019124, metrics={'train_runtime': 3089.0213, 'train_samples_per_second': 25.085, 'train_steps_per_second': 5.019, 'total_flos': 2.425995315478364e+17, 'train_loss': 1.8674790621019124, 'epoch': 8.0})

In [ ]:
model.save_pretrained(training_args.output_dir)

In [ ]:
import os
os.listdir('output')

['adapter_model.bin',
 'runs',
 'adapter_config.json',
 'checkpoint-14000',
 'checkpoint-12000']

In [ ]:
from shutil import rmtree
from distutils.dir_util import copy_tree

In [ ]:
copy_tree("output", "/content/drive/MyDrive/twitter/output")

['/content/drive/MyDrive/twitter/output/adapter_model.bin',
 '/content/drive/MyDrive/twitter/output/runs/Apr16_16-02-29_13a55f4ebd4e/1681661005.7692783/events.out.tfevents.1681661005.13a55f4ebd4e.5711.1',
 '/content/drive/MyDrive/twitter/output/runs/Apr16_16-02-29_13a55f4ebd4e/events.out.tfevents.1681661005.13a55f4ebd4e.5711.0',
 '/content/drive/MyDrive/twitter/output/runs/Apr16_16-06-46_13a55f4ebd4e/1681661245.260394/events.out.tfevents.1681661245.13a55f4ebd4e.7612.1',
 '/content/drive/MyDrive/twitter/output/runs/Apr16_16-06-46_13a55f4ebd4e/events.out.tfevents.1681661245.13a55f4ebd4e.7612.0',
 '/content/drive/MyDrive/twitter/output/adapter_config.json',
 '/content/drive/MyDrive/twitter/output/checkpoint-14000/adapter_model.bin',
 '/content/drive/MyDrive/twitter/output/checkpoint-14000/rng_state.pth',
 '/content/drive/MyDrive/twitter/output/checkpoint-14000/scheduler.pt',
 '/content/drive/MyDrive/twitter/output/checkpoint-14000/training_args.bin',
 '/content/drive/MyDrive/twitter/outpu

# Eval

In [ ]:
%pip install gradio

In [ ]:
import os
import sys


def fix_pytorch_int8():
    valid_path = [p for p in sys.path if p and os.path.isdir(p)]

    for path in valid_path:
        for folder in os.listdir(path):
            if 'torch' in folder:
                packages_path = path
                break

    fix_path = f'{packages_path}/torch/nn/parameter.py'

    with open(fix_path, 'r') as f:
        text = f.read()

    if 'if data.dtype == torch.int8' not in text:
        text = text.replace(
            '            return torch.Tensor._make_subclass(cls, data, requires_grad)', 
            '            if data.dtype == torch.int8:\n' \
            '                requires_grad = False\n' \
            '            return torch.Tensor._make_subclass(cls, data, requires_grad)'
        )
        with open(fix_path, 'w') as f:
            f.write(text)

        return print('Fixed torch/nn/parameter.py')

In [ ]:
fix_pytorch_int8()

In [ ]:
%cd Fujisaki

/content/Fujisaki


In [ ]:
import torch
import sys
from transformers import AutoTokenizer, GenerationConfig, AutoModel

## CUDA

### FP16

In [ ]:
### FP16

# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "THUDM/chatglm-6b",
    trust_remote_code=True,
    device_map='auto',
    revision='4de8efe'
).half().cuda()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

from peft import get_peft_model, LoraConfig, TaskType, PeftModel
peft_path = 'KumaTea/twitter'  # change it to your own
model = PeftModel.from_pretrained(
       model,
       peft_path,
       torch_dtype=torch.float16
    )

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### INT8

In [ ]:
### INT8

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "KumaTea/twitter-int8",  # change it to your own
    trust_remote_code=True,
    device_map='auto',
    revision="1136001"
).half().cuda()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

### INT4

In [ ]:
### INT4

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "KumaTea/twitter-int4",  # change it to your own
    trust_remote_code=True,
    device_map='auto',
    revision="e2aecb2"
).half().cuda()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

## NO GPU

Main difference:

Remove `device_map='auto'`

### FP16

In [ ]:
### FP16

# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "THUDM/chatglm-6b",
    trust_remote_code=True,
    revision='4de8efe'
).float()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

from peft import get_peft_model, LoraConfig, TaskType, PeftModel
peft_path = 'KumaTea/twitter'  # change it to your own
model = PeftModel.from_pretrained(
       model,
       peft_path,
       torch_dtype=torch.float16
    )

### INT8

In [ ]:
### INT8

# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "KumaTea/twitter-int8",  # change it to your own
    trust_remote_code=True,
    revision="1136001"
).float()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

### INT4

In [ ]:
### INT4

# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "int4",
    trust_remote_code=True,
    revision="e2aecb2"
).float()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

## Infer

In [ ]:
# torch.set_default_tensor_type(torch.cuda.FloatTensor)
# torch.set_default_tensor_type(torch.FloatTensor)
model.eval()

In [ ]:
def evaluate(context, temperature, top_p, top_k):
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        #repetition_penalty=1.1,
        num_beams=1,
        do_sample=True,
    )
    with torch.no_grad():
        input_text = f"Context: {context}Answer: " 
        ids = tokenizer([input_text], return_tensors="pt")
        inputs = ids.to("cuda")
        out = model.generate(
            **inputs,
            max_length=224,
            generation_config=generation_config
        )
        out = out.tolist()[0]
        decoder_output = tokenizer.decode(out)
        out_text = decoder_output.split("Answer: ")[1]
        return out_text


def evaluate_stream(msg, history, temperature, top_p):
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        #repetition_penalty=1.1,
        num_beams=1,
        do_sample=True,
    )

    history.append([msg, None])

    context = ""
    if len(history) > 4:
        history.pop(0)

    for j in range(len(history)):
        history[j][0] = history[j][0].replace("<br>", "")

    # concatenate context
    for h in history[:-1]:
        context += h[0] + "||" + h[1] + "||"

    context += history[-1][0]
    context = context.replace(r'<br>', '')

    # TODO: Avoid the tokens are too long.
    CUTOFF = 224
    while len(tokenizer.encode(context)) > CUTOFF:
        # save 15 token size for the answer
        context = context[15:]

    h = []
    print("History:", history)
    print("Context:", context)
    for response, h in model.stream_chat(tokenizer, context, h, max_length=CUTOFF, top_p=top_p, temperature=temperature):
        history[-1][1] = response
        yield history, ""

In [ ]:
history = [['你是谁','我是 kuma'],]
r = []
for h in evaluate_stream("你在干什么", history, 1.0, 0.9):
    r.append(h)
print(r[-1][0][-1][-1])

History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我在做实验，这是生物专业的(
0.5824484310005573
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我在干什么不知道，我在玩游戏 🥺🥺🥺
1.2787180160003118
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我并没有在工作或者学习 😗
是玩游戏的 😗😗😗
1.7947714659994745
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我昨天刚刚成年，所以是kuma的监护人
0.6862346529997012
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我是kuma(
我是猫娘！🥰
而且我是大佬！🤤
1.3786690090000775
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
在摸我的头 😋
0.6267644339995968
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我本来打算今天回答你的一个提问的(
0.7048833489998287
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
在玩游戏
0.1911847259998467
History: [['你是谁', '我是 kuma'], ['你在干什么', None]]
Context: 你是谁||我是 kuma||你在干什么
我准备起床了 🥺
0.6388007280002057
His

## Interactive

In [ ]:
import gradio as gr

title = """<h1 align="center">李萌萌（Alter Ego）</h1>
<h3 align="center">这是一个通过ChatGLM模型训练的李萌萌的数字分身，你可以与她聊天，或者直接在文本框按下Enter，来观察李萌萌到底会说些什么。</h3>"""

footer =  """<p align='center'>项目在<a href='https://github.com/ljsabc/Fujisaki' target='_blank'>GitHub</a>上托管，基于清华的<a href='https://huggingface.co/THUDM/chatglm-6b' target='_blank'>THUDM/chatglm-6b</a>项目。</p>
<p align='center'><em>"I'm... a boy." --Chihiro Fujisaki</em></p>"""

with gr.Blocks() as demo:
    gr.HTML(title)
    state = gr.State()
    with gr.Row():
        with gr.Column(scale=2):
            temp = gr.components.Slider(minimum=0, maximum=1.1, value=0.95, label="Temperature",
                info="温度参数，越高的温度生成的内容越丰富，但是有可能出现语法问题。")
            top_p = gr.components.Slider(minimum=0.5, maximum=1.0, value=0.99, label="Top-p",
                info="top-p参数，只输出前p>top-p的文字，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。")
            #code = gr.Textbox(label="temp_output", info="解码器输出")
            #top_k = gr.components.Slider(minimum=1, maximum=200, step=1, value=25, label="Top k",
            #    info="top-k参数，下一个输出的文字会从top-k个文字中进行选择，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。")
            
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="聊天框", info="")
            msg = gr.Textbox(label="输入框", placeholder="最近过得怎么样？",
                info="输入你的内容，按[Enter]发送。也可以什么都不填写生成随机数据。")
            clear = gr.Button("清除聊天")

    msg.submit(evaluate_stream, [msg, chatbot, temp, top_p], [chatbot, msg])
    clear.click(lambda: None, None, chatbot, queue=False)
    gr.HTML(footer)

demo.queue()
demo.launch(debug=False)


In [ ]:
import gradio as gr
gr.Interface(
    fn=evaluate,
    inputs=[
        gr.components.Textbox(
            lines=2, label="问题", placeholder="最近过得怎么样？",
            info="可以在这里输入你的问题。也可以什么都不填写生成随机数据。"
        ),
        #gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(minimum=0, maximum=1.1, value=1.0, label="Temperature",
            info="温度参数，越高的温度生成的内容越丰富，但是有可能出现语法问题。"),
        gr.components.Slider(minimum=0.5, maximum=1.0, value=0.99, label="Top p",
            info="top-p参数，只输出前p>top-p的文字，建议不要修改。"),
        gr.components.Slider(minimum=1, maximum=200, step=1, value=25, label="Top k",
            info="top-k参数，下一个输出的文字会从top-k个文字中进行选择，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。"),
    ],
    outputs=[
        gr.inputs.Textbox(
            lines=5,
            label="Output",
        )
    ],
    title="李萌萌（Alter Ego）",
    description="这是一个通过ChatGLM模型训练的李萌萌的数字分身，你可以在问题栏目填入内容，或者什么都不填，来观察李萌萌到底会说些什么。",
).launch()

# Quantize

In [ ]:
%cd Fujisaki

/content/Fujisaki


In [ ]:
fix_pytorch_int8()

In [ ]:
import torch
import sys
from transformers import AutoTokenizer, GenerationConfig, AutoModel

In [ ]:
### FP16

# torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained(
    "THUDM/chatglm-6b",
    trust_remote_code=True,
    device_map='auto',
    revision='4de8efe'
).cuda()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision='4de8efe')

from peft import get_peft_model, LoraConfig, TaskType, PeftModel
peft_path = 'KumaTea/twitter'      # change it to your own
model = PeftModel.from_pretrained(
       model,
       peft_path,
       torch_dtype=torch.float16
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

In [ ]:
# torch.set_default_tensor_type(torch.cuda.FloatTensor)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(130528, 4096)
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=12288, bias=False)
                )
              )
              (dense): Linear(in_features=4096, out_features=40

In [ ]:
type(model)

peft.peft_model.PeftModelForCausalLM

**WHAT'S AMAZING**

↓↓↓

In [ ]:
model = model.merge_and_unload()

In [ ]:
type(model)

transformers_modules.THUDM.chatglm-6b.4de8efebc837788ffbfc0a15663de8553da362a2.modeling_chatglm.ChatGLMForConditionalGeneration

In [ ]:
model_int8 = model.quantize(8).half().cuda()

In [ ]:
model_int8.save_pretrained('output/int8/')

In [ ]:
!ls -lh output/int8/

total 6.3G
-rw-r--r-- 1 root root  870 Apr 16 17:16 config.json
-rw-r--r-- 1 root root 4.2K Apr 16 17:16 configuration_chatglm.py
-rw-r--r-- 1 root root  142 Apr 16 17:16 generation_config.json
-rw-r--r-- 1 root root  57K Apr 16 17:16 modeling_chatglm.py
-rw-r--r-- 1 root root 6.3G Apr 16 17:16 pytorch_model.bin
-rw-r--r-- 1 root root  15K Apr 16 17:16 quantization.py


In [ ]:
model_int8 is model

True

In [ ]:
model_int4 = model.quantize(4).half().cuda()

In [ ]:
model.save_pretrained('output/int4/')

In [ ]:
!ls -lh output/int4/

total 3.7G
-rw-r--r-- 1 root root  870 Apr 16 17:17 config.json
-rw-r--r-- 1 root root 4.2K Apr 16 17:17 configuration_chatglm.py
-rw-r--r-- 1 root root  142 Apr 16 17:17 generation_config.json
-rw-r--r-- 1 root root  57K Apr 16 17:17 modeling_chatglm.py
-rw-r--r-- 1 root root 3.7G Apr 16 17:17 pytorch_model.bin
-rw-r--r-- 1 root root  15K Apr 16 17:17 quantization.py


In [ ]:
model_int4 is model

True

In [ ]:
!cp -av output/int8 /content/drive/MyDrive/twitter/quantized/
!cp -av output/int4 /content/drive/MyDrive/twitter/quantized/

'output/int8/modeling_chatglm.py' -> '/content/drive/MyDrive/twitter/quantized/int8/modeling_chatglm.py'
'output/int8/quantization.py' -> '/content/drive/MyDrive/twitter/quantized/int8/quantization.py'
'output/int8/configuration_chatglm.py' -> '/content/drive/MyDrive/twitter/quantized/int8/configuration_chatglm.py'
'output/int8/config.json' -> '/content/drive/MyDrive/twitter/quantized/int8/config.json'
'output/int8/generation_config.json' -> '/content/drive/MyDrive/twitter/quantized/int8/generation_config.json'
'output/int8/pytorch_model.bin' -> '/content/drive/MyDrive/twitter/quantized/int8/pytorch_model.bin'
'output/int4/modeling_chatglm.py' -> '/content/drive/MyDrive/twitter/quantized/int4/modeling_chatglm.py'
'output/int4/configuration_chatglm.py' -> '/content/drive/MyDrive/twitter/quantized/int4/configuration_chatglm.py'
'output/int4/quantization.py' -> '/content/drive/MyDrive/twitter/quantized/int4/quantization.py'
'output/int4/config.json' -> '/content/drive/MyDrive/twitter/quan